In [1]:
import requests
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from IPython.display import clear_output

In [2]:
CLIENT_ACCESS_TOKEN = "lTv0ZCyw1mkXQ7_9bZ9GmEdc9vPh2_VQzhFCpERSOYiFA1fXjEVrI8BY-D9_k6E2"
BASE_URI = "https://api.genius.com"

In [6]:
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URI, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

print("searching LPDA's artist id. \n")

# find artist id from given data.
find_id = _get("search", {'q': 'Las Pastillas Del Abuelo'})
for hit in find_id["response"]["hits"]:
   if hit["result"]["primary_artist"]["name"] == 'Las Pastillas Del Abuelo':
       artist_id = hit["result"]["primary_artist"]["id"]
       break

print("-> LPDA's id is " + str(artist_id) + "\n")

searching LPDA's artist id. 

-> LPDA's id is 347470



In [7]:
LPDA_ID = 347470

In [8]:
def _get(path, params=None, headers=None):

    url = '/'.join([BASE_URI, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}
    response = requests.get(url=url, params=params, headers=headers)
    response.raise_for_status()

    return response.json()


def get_artist_songs(artist_id):

    current_page = 1
    next_page = True
    songs = []

    while next_page:
        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page}
        data = _get(path=path, params=params)
        page_songs = data['response']['songs']
        if page_songs:
            songs += page_songs
            current_page += 1
        else:
            next_page = False

    return songs

In [9]:
songs = get_artist_songs(LPDA_ID)
songs = [song for song in songs if song['primary_artist']['id'] == LPDA_ID]

songs

[{'annotation_count': 0,
  'api_path': '/songs/3823082',
  'full_title': 'Absolutismos by\xa0Las\xa0Pastillas Del Abuelo',
  'header_image_thumbnail_url': 'https://images.genius.com/5cbaa1db8d671abe3140e90ba56c173a.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/5cbaa1db8d671abe3140e90ba56c173a.953x953x1.jpg',
  'id': 3823082,
  'lyrics_owner_id': 6437626,
  'lyrics_state': 'complete',
  'path': '/Las-pastillas-del-abuelo-absolutismos-lyrics',
  'pyongs_count': None,
  'song_art_image_thumbnail_url': 'https://images.genius.com/5cbaa1db8d671abe3140e90ba56c173a.300x300x1.jpg',
  'song_art_image_url': 'https://images.genius.com/5cbaa1db8d671abe3140e90ba56c173a.953x953x1.jpg',
  'stats': {'unreviewed_annotations': 0, 'hot': False},
  'title': 'Absolutismos',
  'title_with_featured': 'Absolutismos',
  'url': 'https://genius.com/Las-pastillas-del-abuelo-absolutismos-lyrics',
  'primary_artist': {'api_path': '/artists/347470',
   'header_image_url': 'https://images.genius.com

In [10]:
def scrape_lyrics(url):

    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')
    lyrics = soup.find("div", {"class": "lyrics"})
    lyrics = cleanhtml(str(lyrics))
    return lyrics

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

total = len(songs)
for i, song in enumerate(songs):
    if 0 == i % 10:
        clear_output(wait=True)
        print("%d songs parsed out of %d" % (i, total))
    url = song['url']
    lyrics = scrape_lyrics(url)
    song['lyrics'] = lyrics

130 songs parsed out of 137


In [11]:
df = pd.read_json(json.dumps(songs))
df.to_csv('lpda_raw_songs.csv', sep=';',index=False)

In [12]:
df.head()

,annotation_count,api_path,full_title,header_image_thumbnail_url,header_image_url,id,lyrics_owner_id,lyrics_state,path,pyongs_count,song_art_image_thumbnail_url,song_art_image_url,stats,title,title_with_featured,url,primary_artist,lyrics
0,0,/songs/3823082,Absolutismos by Las Pastillas Del Abuelo,https://images.genius.com/5cbaa1db8d671abe3140...,https://images.genius.com/5cbaa1db8d671abe3140...,3823082,6437626,complete,/Las-pastillas-del-abuelo-absolutismos-lyrics,NaN,https://images.genius.com/5cbaa1db8d671abe3140...,https://images.genius.com/5cbaa1db8d671abe3140...,"{'unreviewed_annotations': 0, 'hot': False}",Absolutismos,Absolutismos,https://genius.com/Las-pastillas-del-abuelo-ab...,"{'api_path': '/artists/347470', 'header_image_...","\n\n[Letra de ""Absolutismos""]\n\nPerdónenme si..."
1,1,/songs/750807,Algo de Vos by Las Pastillas Del Abuelo,https://images.genius.com/0ac61135be634f795a40...,https://images.genius.com/0ac61135be634f795a40...,750807,6974707,complete,/Las-pastillas-del-abuelo-algo-de-vos-lyrics,NaN,https://images.genius.com/0ac61135be634f795a40...,https://images.genius.com/0ac61135be634f795a40...,"{'unreviewed_annotations': 0, 'hot': False}",Algo de Vos,Algo de Vos,https://genius.com/Las-pastillas-del-abuelo-al...,"{'api_path': '/artists/347470', 'header_image_...","\n\n[Letra de ""Algo de Vos""]\n\n[Verso 1]\nHe ..."
2,0,/songs/4678749,Algo de Vos (demo) by Las Pastillas Del Abuelo,https://images.genius.com/fdcc7d0a50637d8b97d9...,https://images.genius.com/fdcc7d0a50637d8b97d9...,4678749,6974707,complete,/Las-pastillas-del-abuelo-algo-de-vos-demo-lyrics,NaN,https://images.genius.com/fdcc7d0a50637d8b97d9...,https://images.genius.com/fdcc7d0a50637d8b97d9...,"{'unreviewed_annotations': 0, 'hot': False}",Algo de Vos (demo),Algo de Vos (demo),https://genius.com/Las-pastillas-del-abuelo-al...,"{'api_path': '/artists/347470', 'header_image_...","\n\n[Letra de ""Algo de Vos (demo)""]\n\n[Verso ..."
3,0,/songs/3823057,Almafuerte by Las Pastillas Del Abuelo,https://images.genius.com/0ac61135be634f795a40...,https://images.genius.com/0ac61135be634f795a40...,3823057,6974707,complete,/Las-pastillas-del-abuelo-almafuerte-lyrics,NaN,https://images.genius.com/0ac61135be634f795a40...,https://images.genius.com/0ac61135be634f795a40...,"{'unreviewed_annotations': 0, 'hot': False}",Almafuerte,Almafuerte,https://genius.com/Las-pastillas-del-abuelo-al...,"{'api_path': '/artists/347470', 'header_image_...","\n\n[Letra de ""Almafuerte""]\n\n[Verso]\nYo sé ..."
4,0,/songs/4677441,Almagro Haedo by Las Pastillas Del Abuelo,https://images.genius.com/0ac61135be634f795a40...,https://images.genius.com/0ac61135be634f795a40...,4677441,6974707,complete,/Las-pastillas-del-abuelo-almagro-haedo-lyrics,NaN,https://images.genius.com/0ac61135be634f795a40...,https://images.genius.com/0ac61135be634f795a40...,"{'unreviewed_annotations': 0, 'hot': False}",Almagro Haedo,Almagro Haedo,https://genius.com/Las-pastillas-del-abuelo-al...,"{'api_path': '/artists/347470', 'header_image_...","\n\n[Letra de ""Almagro Haedo""]\n\n[Verso 1]\nM..."


In [13]:
#Eliminamos info inutil

In [16]:
del df['api_path']
del df['full_title']
del df['header_image_thumbnail_url']
del df['header_image_url']
del df['lyrics_owner_id']
del df['path']
del df['primary_artist']
del df['pyongs_count']
del df['song_art_image_thumbnail_url']
del df['stats']
#wow, almost all information is useless
del df['title_with_featured']
del df['lyrics_state']



KeyError: 'api_path'

In [17]:
df.head()

,annotation_count,id,song_art_image_url,title,url,lyrics
0,0,3823082,https://images.genius.com/5cbaa1db8d671abe3140...,Absolutismos,https://genius.com/Las-pastillas-del-abuelo-ab...,"\n\n[Letra de ""Absolutismos""]\n\nPerdónenme si..."
1,1,750807,https://images.genius.com/0ac61135be634f795a40...,Algo de Vos,https://genius.com/Las-pastillas-del-abuelo-al...,"\n\n[Letra de ""Algo de Vos""]\n\n[Verso 1]\nHe ..."
2,0,4678749,https://images.genius.com/fdcc7d0a50637d8b97d9...,Algo de Vos (demo),https://genius.com/Las-pastillas-del-abuelo-al...,"\n\n[Letra de ""Algo de Vos (demo)""]\n\n[Verso ..."
3,0,3823057,https://images.genius.com/0ac61135be634f795a40...,Almafuerte,https://genius.com/Las-pastillas-del-abuelo-al...,"\n\n[Letra de ""Almafuerte""]\n\n[Verso]\nYo sé ..."
4,0,4677441,https://images.genius.com/0ac61135be634f795a40...,Almagro Haedo,https://genius.com/Las-pastillas-del-abuelo-al...,"\n\n[Letra de ""Almagro Haedo""]\n\n[Verso 1]\nM..."


In [18]:
del df['annotation_count']
del df['song_art_image_url']

In [19]:
df.head()

,id,title,url,lyrics
0,3823082,Absolutismos,https://genius.com/Las-pastillas-del-abuelo-ab...,"\n\n[Letra de ""Absolutismos""]\n\nPerdónenme si..."
1,750807,Algo de Vos,https://genius.com/Las-pastillas-del-abuelo-al...,"\n\n[Letra de ""Algo de Vos""]\n\n[Verso 1]\nHe ..."
2,4678749,Algo de Vos (demo),https://genius.com/Las-pastillas-del-abuelo-al...,"\n\n[Letra de ""Algo de Vos (demo)""]\n\n[Verso ..."
3,3823057,Almafuerte,https://genius.com/Las-pastillas-del-abuelo-al...,"\n\n[Letra de ""Almafuerte""]\n\n[Verso]\nYo sé ..."
4,4677441,Almagro Haedo,https://genius.com/Las-pastillas-del-abuelo-al...,"\n\n[Letra de ""Almagro Haedo""]\n\n[Verso 1]\nM..."


In [20]:
df['lyrics'].replace(regex=True,inplace=True,to_replace=r'(\n)+',value=r'\n')
df['lyrics'].replace(regex=True,inplace=True,to_replace=r'^(\n)+',value=r'')

In [21]:
df.to_csv('lpda_fixed.csv', sep=';',index=False)

In [23]:
df.head()

,id,title,url,lyrics
0,3823082,Absolutismos,https://genius.com/Las-pastillas-del-abuelo-ab...,"[Letra de ""Absolutismos""]\nPerdónenme si voy a..."
1,750807,Algo de Vos,https://genius.com/Las-pastillas-del-abuelo-al...,"[Letra de ""Algo de Vos""]\n[Verso 1]\nHe limpia..."
2,4678749,Algo de Vos (demo),https://genius.com/Las-pastillas-del-abuelo-al...,"[Letra de ""Algo de Vos (demo)""]\n[Verso 1]\nHe..."
3,3823057,Almafuerte,https://genius.com/Las-pastillas-del-abuelo-al...,"[Letra de ""Almafuerte""]\n[Verso]\nYo sé que fi..."
4,4677441,Almagro Haedo,https://genius.com/Las-pastillas-del-abuelo-al...,"[Letra de ""Almagro Haedo""]\n[Verso 1]\nMe abri..."


In [24]:
#Manual corrections
df.nunique(), df.shape


(id        137
 title     137
 url       137
 lyrics    136
 dtype: int64, (137, 4))

In [25]:
df[df.duplicated(subset=['lyrics'])]


,id,title,url,lyrics
105,1963441,Perdido - en vivo,https://genius.com/Las-pastillas-del-abuelo-pe...,Hoy les voy a contar\nUn poco mi forma de ser\...


In [26]:
indexes_to_drop = df[df.duplicated(subset=['lyrics'])].index


In [27]:
df = df.drop(indexes_to_drop).reset_index(drop=True)


In [28]:
df.drop(['url', 'id'], axis=1)


,title,lyrics
0,Absolutismos,"[Letra de ""Absolutismos""]\nPerdónenme si voy a..."
1,Algo de Vos,"[Letra de ""Algo de Vos""]\n[Verso 1]\nHe limpia..."
2,Algo de Vos (demo),"[Letra de ""Algo de Vos (demo)""]\n[Verso 1]\nHe..."
3,Almafuerte,"[Letra de ""Almafuerte""]\n[Verso]\nYo sé que fi..."
4,Almagro Haedo,"[Letra de ""Almagro Haedo""]\n[Verso 1]\nMe abri..."
...,...,...
131,Viejo,"[Letra de ""Viejo""]\n[Verso 1]\nQuien diga que ..."
132,Viejo Karma!,"Cómo te extraño nena, no sabes\nLo embobado qu..."
133,Viejo karma - en vivo,Cómo te extraño nena\nNo sabes lo embobado que...
134,Viles medios!,Diciendo más en los medios que en mis cancione...


In [29]:
df.to_csv('lpda_fixed_2.csv', sep=';',index=False)
